In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
# 1. 데이터 불러오기
df = pd.read_csv("../../data/MED-PHCENTER/MED-PHCENTER.csv")

# 2. "-" → 0 치환 후 숫자 변환
for col in ["보건소", "보건분소", "보건지소"]:
    df[col] = df[col].replace("-", 0)
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)

# 3. 소계 행 제거
df = df[df["자치구"] != "소계"]

# 4. 자치구별 합산
agg = df.groupby("자치구")[["보건소", "보건분소", "보건지소"]].sum().reset_index()

# 5. Z-Score 정규화 → MinMax Scaling (0~100)
scaler_z = StandardScaler()
scaler_mm = MinMaxScaler()

z_scaled = scaler_z.fit_transform(agg[["보건소", "보건분소", "보건지소"]])
mm_scaled = scaler_mm.fit_transform(z_scaled)

agg[["phcenter_score", "phbranch_score", "phsubcenter_score"]] = mm_scaled * 100

# 6. 평균 점수
agg["phcenter_mean_score"] = agg[["phcenter_score", "phbranch_score", "phsubcenter_score"]].mean(axis=1)

# 7. 저장
agg.to_csv("../../result/MED/MED-PHCENTER_result.csv", index=False, encoding="utf-8-sig")

print(agg.head())

   자치구  보건소  보건분소  보건지소  phcenter_score  phbranch_score  phsubcenter_score  \
0  강남구    2     2     2             0.0            50.0               25.0   
1  강동구    2     0     4             0.0             0.0               50.0   
2  강북구    2     0     4             0.0             0.0               50.0   
3  강서구    2     2     4             0.0            50.0               50.0   
4  관악구    2     2     2             0.0            50.0               25.0   

   phcenter_mean_score  
0            25.000000  
1            16.666667  
2            16.666667  
3            33.333333  
4            25.000000  
